# How to set a prior given T=1 nonzero effect?

Assume one nonzero effect $T = 1$. We investigate SuSiE's performance when we have a pve such that SuSiE has certain amount of power e.g. $pve = 0.05$.

## Results

* If we overspecify a prior, i.e. set a prior $> 0.05$, we would have higher power and 0 fdr. But the compensation is that the size of confidence set becomes larger. Also, if we take the top hit in each confidence set, we have $12\%$ probability getting wrong. 

* If we underspecify a prior, i.e. set a prior $< 0.05$, we have lower power compared to overspecification, and 0 fdr. The size of confidence set is right being 1. And the top-hit-rate is $100\%$. 

The next question we want to ask is: what if the number of nonzero effects is not 1? Should we overspecify or underspecify a prior? 



In [7]:
dscout.summary[dscout.summary$pve==0.05,]

,effect_num,pve,prior,power,fdr,cs_size,top_hit_rate,cs_num
19,1,0.05,0.01,0.7,0,1.000000,1.0000,0.7
85,1,0.05,0.02,0.7,0,1.000000,1.0000,0.7
151,1,0.05,0.03,0.7,0,1.000000,1.0000,0.7
217,1,0.05,0.05,0.7,0,1.000000,1.0000,0.7
283,1,0.05,0.10,0.9,0,6.000000,0.8889,0.9
349,1,0.05,0.20,0.9,0,5.333333,0.8889,0.9
415,1,0.05,0.40,0.9,0,5.000000,0.8889,0.9
481,1,0.05,0.50,0.9,0,4.888889,0.8889,0.9
547,1,0.05,0.70,0.9,0,4.888889,0.8889,0.9
613,1,0.05,0.90,0.9,0,4.777778,0.8889,0.9


## Code details

In [1]:
dscout_Q2 = readRDS('dscout_Q2.rds')
dscout_Q2 = dscout_Q2[!is.na(dscout_Q2$sim_gaussian.output.file),]
dscout_Q2 = dscout_Q2[!is.na(dscout_Q2$susie_prior.output.file),]

In [2]:
dscout_df = data.frame(dscout_Q2$sim_gaussian.effect_num, dscout_Q2$sim_gaussian.pve, dscout_Q2$susie_prior.prior,
                       dscout_Q2$score.hit, dscout_Q2$score.signal_num, dscout_Q2$score.cs_medianSize,
                       dscout_Q2$score.top_hit, dscout_Q2$sim_gaussian.mean_corX, dscout_Q2$susie_prior.avg_purity)
names(dscout_df) = c('effect_num', 'pve', 'prior','hit', 'cs_num', 'cs_size', 'top_hit', 'corX', 'avg_purity')

In [3]:
power.summary = aggregate(hit ~ effect_num + pve + prior, dscout_df, sum)
power.summary$power = power.summary$hit / (power.summary$effect_num*10)
fdr.summary = aggregate(cs_num ~ effect_num + pve + prior, dscout_df, sum)
fdr.summary$fdr = round(1 - power.summary$hit / fdr.summary$cs_num, 4)
meannonzero = function(x){mean(x[x!=0])}
setsize.summary = aggregate(cs_size ~ effect_num + pve + prior, dscout_df, meannonzero)
tophit.summary = aggregate(top_hit ~ effect_num + pve + prior, dscout_df, sum)
tophit.summary$tophit_rate = round(tophit.summary$top_hit / fdr.summary$cs_num , 4)

In [4]:
dscout.summary = data.frame(power.summary$effect_num, power.summary$pve, power.summary$prior,
                            power.summary$power, fdr.summary$fdr, setsize.summary$cs_size, tophit.summary$tophit_rate,
                           fdr.summary$cs_num/10)
names(dscout.summary) = c('effect_num', 'pve', 'prior','power', 'fdr', 'cs_size', 'top_hit_rate', 'cs_num')

In [5]:
is.nan.data.frame <- function(x)
do.call(cbind, lapply(x, is.nan))
dscout.summary[is.nan(dscout.summary)] = 0

In [6]:
dscout.summary = dscout.summary[dscout.summary$effect_num==1, ]